##### Copyright 2018 The TensorFlow Hub Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
# Copyright 2018 The TensorFlow Hub Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

# TF-Hub による簡単なテキスト分類器の構築方法


> 注意: このチュートリアルでは、**非推奨**の TensorFlow 1 の機能を使用しています。このタスクの最新アプローチについては、[TensorFlow 2 バージョン](https://www.tensorflow.org/hub/tutorials/tf2_text_classification)をご覧ください。


<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ja/hub/tutorials/text_classification_with_tf_hub.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Run in Google Colab</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ja/hub/tutorials/text_classification_with_tf_hub.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub でソースを表示</a></td>
  <td>     <a href="https://tfhub.dev/google/nnlm-en-dim128/1"><img src="https://www.tensorflow.org/images/hub_logo_32px.png">	TF Hub モデルを参照</a>
</td>
</table>

TensorFlow Hub (TF-Hub) は、機械学習の知識を再利用可能なリソース、特にトレーニング済みの**モジュール**で共有するためのプラットフォームです。このチュートリアルは次の 2 つの主要部分で構成しています。

**入門編:** TF-Hub によるテキスト分類器のトレーニング

TF-Hub のテキスト埋め込みモジュールを使用して、適切なベースライン精度を持つ単純な感情分類器をトレーニングします。その後、予測を分析してモデルが適切であるかを確認し、精度を向上させるための改善点を提案します。

**上級編:** 転移学習の分析

この項目では、様々な TF-Hub モジュールを使用して Estimator の精度への効果を比較し、転移学習のメリットとデメリットを実証します。


## オプションの前提条件

- Tensorflow の[既製の Estimator フレームワーク](https://www.tensorflow.org/get_started/premade_estimators)に対する基礎知識があること。
- [Pandas](https://pandas.pydata.org/) ライブラリを熟知していること。


## セットアップ

In [ ]:
# Install TF-Hub.
!pip install seaborn

Tensorflow のインストールに関する詳細は、[https://www.tensorflow.org/install/](https://www.tensorflow.org/install/) をご覧ください。

In [ ]:
from absl import logging

import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

# はじめに

## データ

[映画レビューの大規模データセット v1.0](http://ai.stanford.edu/~amaas/data/sentiment/) タスク [(Mass et al., 2011)](http://ai.stanford.edu/~amaas/papers/wvSent_acl2011.pdf) を解決してみます。データセットは、1 から 10 までの肯定度でラベル付けされた IMDB 映画レビューで構成されています。タスクは、レビューを**ネガティブ (negative)** または**ポジティブ (positive)** にラベル付けすることです。

In [ ]:
# Load all files from a directory in a DataFrame.
def load_directory_data(directory):
  data = {}
  data["sentence"] = []
  data["sentiment"] = []
  for file_path in os.listdir(directory):
    with tf.io.gfile.GFile(os.path.join(directory, file_path), "r") as f:
      data["sentence"].append(f.read())
      data["sentiment"].append(re.match("\d+_(\d+)\.txt", file_path).group(1))
  return pd.DataFrame.from_dict(data)

# Merge positive and negative examples, add a polarity column and shuffle.
def load_dataset(directory):
  pos_df = load_directory_data(os.path.join(directory, "pos"))
  neg_df = load_directory_data(os.path.join(directory, "neg"))
  pos_df["polarity"] = 1
  neg_df["polarity"] = 0
  return pd.concat([pos_df, neg_df]).sample(frac=1).reset_index(drop=True)

# Download and process the dataset files.
def download_and_load_datasets(force_download=False):
  dataset = tf.keras.utils.get_file(
      fname="aclImdb.tar.gz", 
      origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
      extract=True)
  
  train_df = load_dataset(os.path.join(os.path.dirname(dataset), 
                                       "aclImdb", "train"))
  test_df = load_dataset(os.path.join(os.path.dirname(dataset), 
                                      "aclImdb", "test"))
  
  return train_df, test_df

# Reduce logging output.
logging.set_verbosity(logging.ERROR)

train_df, test_df = download_and_load_datasets()
train_df.head()

## モデル

### 入力関数

[Estimator フレームワーク](https://www.tensorflow.org/get_started/premade_estimators#overview_of_programming_with_estimators)は、Pandasの データフレームをラップする[入力関数](https://www.tensorflow.org/api_docs/python/tf/compat/v1/estimator/inputs/pandas_input_fn)を提供します。

In [ ]:
# Training input on the whole training set with no limit on training epochs.
train_input_fn = tf.compat.v1.estimator.inputs.pandas_input_fn(
    train_df, train_df["polarity"], num_epochs=None, shuffle=True)

# Prediction on the whole training set.
predict_train_input_fn = tf.compat.v1.estimator.inputs.pandas_input_fn(
    train_df, train_df["polarity"], shuffle=False)
# Prediction on the test set.
predict_test_input_fn = tf.compat.v1.estimator.inputs.pandas_input_fn(
    test_df, test_df["polarity"], shuffle=False)

### 特徴量カラム

TF-Hub は、特定のテキスト特徴量にモジュールを適用し、モジュールの出力をさらに渡す、[特徴量カラム](https://www.tensorflow.org/hub/api_docs/python/hub/text_embedding_column.md)を提供しています。このチュートリアルでは [nlm-en-dim128 モジュール](https://tfhub.dev/google/nnlm-en-dim128/1)を使用します。このチュートリアルにおいて、最も重要な事実は次の通りです。

- このモジュールは、**文のバッチを1 次元のテンソル文字列で**入力として受け取ります。
- このモジュールは、**文の前処理**（例えば、句読点の削除やスペースの分割など）を担当します。
- このモジュールは任意の入力に使用できます（例えば **nlm-en-dim128** は、語彙に存在していない単語を 20.000 バケットまでハッシュします）。

In [ ]:
embedded_text_feature_column = hub.text_embedding_column(
    key="sentence", 
    module_spec="https://tfhub.dev/google/nnlm-en-dim128/1")

### Estimator

分類には [DNN 分類器](https://www.tensorflow.org/api_docs/python/tf/estimator/DNNClassifier)を使用することができます。（注: ラベル関数の異なるモデリングに関しては、追加の留意点がチュートリアルの最後にあります。）

In [ ]:
estimator = tf.estimator.DNNClassifier(
    hidden_units=[500, 100],
    feature_columns=[embedded_text_feature_column],
    n_classes=2,
    optimizer=tf.keras.optimizers.Adagrad(lr=0.003))

### トレーニング

妥当なステップ数の分だけ、Estimator をトレーニングします。

In [ ]:
# Training for 5,000 steps means 640,000 training examples with the default
# batch size. This is roughly equivalent to 25 epochs since the training dataset
# contains 25,000 examples.
estimator.train(input_fn=train_input_fn, steps=5000);

# 予測する

トレーニングセットとテストセットの両方で予測を実行します。

In [ ]:
train_eval_result = estimator.evaluate(input_fn=predict_train_input_fn)
test_eval_result = estimator.evaluate(input_fn=predict_test_input_fn)

print("Training set accuracy: {accuracy}".format(**train_eval_result))
print("Test set accuracy: {accuracy}".format(**test_eval_result))

## 混同行列

混同行列を目視確認して、誤分類の分布を把握することができます。

In [ ]:
def get_predictions(estimator, input_fn):
  return [x["class_ids"][0] for x in estimator.predict(input_fn=input_fn)]

LABELS = [
    "negative", "positive"
]

# Create a confusion matrix on training data.
cm = tf.math.confusion_matrix(train_df["polarity"], 
                              get_predictions(estimator, predict_train_input_fn))

# Normalize the confusion matrix so that each row sums to 1.
cm = tf.cast(cm, dtype=tf.float32)
cm = cm / tf.math.reduce_sum(cm, axis=1)[:, np.newaxis]

sns.heatmap(cm, annot=True, xticklabels=LABELS, yticklabels=LABELS);
plt.xlabel("Predicted");
plt.ylabel("True");

# もっと改善するために

1. **感情の回帰**: 極性クラスに各例を割り当てる際には分類器を使用しました。しかし実際には感情という、もう 1 つの自由に使える分類的特徴があります。この場合クラスは実際にはスケールを表し、連続的な範囲に基礎的な値（ポジティブまたはネガティブ）をうまくマッピングすることができます。分類（[DNN 分類器](https://www.tensorflow.org/api_docs/python/tf/contrib/learn/DNNClassifier)）の代わりに回帰（[DNN 回帰器](https://www.tensorflow.org/api_docs/python/tf/contrib/learn/DNNRegressor)）を計算することによって、このプロパティが使用できるようにしました。
2. **大規模モジュール**: 本チュートリアルでは、メモリ使用量を制限するために小さなモジュールを使用しました。もっと語彙が多く埋め込み空間が大きいモジュールを使用すると、精度のポイントがさらに向上する可能性があります。
3. **パラメータ調整**: 学習率やステップ数などのメタパラメータを調整することにより、精度を向上させることができます。特に異なるモジュールを使用している場合にこれは有効です。妥当な結果を得るためには、検証セットが非常に重要な要素です。なぜなら、テストセットに一般化させないでトレーニングデータの予測を学習するモデルを設定するのは非常に簡単だからです。
4. **より複雑なモデル**: 本チュートリアルでは個々の単語を埋め込み、さらに平均値と組み合わせて文の埋め込みを計算するモジュールを使用しました。Sequential モジュール（例えば[ユニバーサルセンテンスエンコーダ](https://tfhub.dev/google/universal-sentence-encoder/2)モジュールなど）を使用して、文の性質をさらに良く捉えることも可能です。あるいは、2 つ以上の TF-Hub モジュールをアンサンブルします。
5. **正則化**: 過適合を防ぐために、[Proximal Adagrad オプティマイザ](https://www.tensorflow.org/api_docs/python/tf/compat/v1/train/ProximalAdagradOptimizer)などの正則化を行うオプティマイザを使用してみるのもよいでしょう。


# 上級編: 転移学習の分析

転移学習によって、**トレーニングリソースが節約され**、<strong>小さなデータセットによるトレーニング</strong>でも良好なモデルの一般化が実現できるようになりました。この項目では、2 つの異なる TF-Hub モジュールを使用してトレーニングを行い、これを実証します。

- **[nnlm-en-dim128](https://tfhub.dev/google/nnlm-en-dim128/1)** - 事前トレーニング済みのテキスト埋め込みモジュール
- **[random-nnlm-en-dim128](https://tfhub.dev/google/random-nnlm-en-dim128/1)** - **nlm-en-dim128** と同じ語彙とネットワークを持ちますが、重みはランダムに初期化され、実際のデータではトレーニングされていない、テキスト埋め込みモジュール

これを 2 つのモードでトレーニングします。

- <strong>分類器のみ</strong>をトレーニングする（つまりモジュールは凍結） 。
- **モジュールと分類器を一緒に**トレーニングする。

様々なモジュールを使用して複数のトレーニングと評価を行い、精度にどのような影響が出るかを見てみましょう。

In [ ]:
def train_and_evaluate_with_module(hub_module, train_module=False):
  embedded_text_feature_column = hub.text_embedding_column(
      key="sentence", module_spec=hub_module, trainable=train_module)

  estimator = tf.estimator.DNNClassifier(
      hidden_units=[500, 100],
      feature_columns=[embedded_text_feature_column],
      n_classes=2,
      optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.003))

  estimator.train(input_fn=train_input_fn, steps=1000)

  train_eval_result = estimator.evaluate(input_fn=predict_train_input_fn)
  test_eval_result = estimator.evaluate(input_fn=predict_test_input_fn)

  training_set_accuracy = train_eval_result["accuracy"]
  test_set_accuracy = test_eval_result["accuracy"]

  return {
      "Training accuracy": training_set_accuracy,
      "Test accuracy": test_set_accuracy
  }


results = {}
results["nnlm-en-dim128"] = train_and_evaluate_with_module(
    "https://tfhub.dev/google/nnlm-en-dim128/1")
results["nnlm-en-dim128-with-module-training"] = train_and_evaluate_with_module(
    "https://tfhub.dev/google/nnlm-en-dim128/1", True)
results["random-nnlm-en-dim128"] = train_and_evaluate_with_module(
    "https://tfhub.dev/google/random-nnlm-en-dim128/1")
results["random-nnlm-en-dim128-with-module-training"] = train_and_evaluate_with_module(
    "https://tfhub.dev/google/random-nnlm-en-dim128/1", True)

結果を見てみましょう。

In [ ]:
pd.DataFrame.from_dict(results, orient="index")

既に複数のパターンが見られますが、まず最初にテストセットのベースラインの精度、つまり最も代表的なクラスのラベルのみを出力して達成可能な下限値を確立させる必要があります。

In [ ]:
estimator.evaluate(input_fn=predict_test_input_fn)["accuracy_baseline"]

最も代表的なクラスを割り当てると、精度は **50%** になります。ここで注目すべき点がいくつかあります。

1. 驚かれるかも知れませんが、**モデルは固定されたランダムな埋め込み上でまだ学習することが可能です**。その理由は、ディクショナリのすべての単語がランダムなベクトルにマップされていたとしても、Estimator は完全に接続されたレイヤーを使用するだけで空間を分離することができるからです。
2. **ランダム埋め込み**を使用したモジュールのトレーニングを許可すると、分類器だけをトレーニングする場合に比べ、トレーニングとテスト両方の精度が向上します。
3. また、**事前トレーニング済みの埋め込み**でモジュールをトレーニングすると、トレーニングとテスト両方の精度が向上します。ただし、トレーニングセットの過適合には注意してください。事前トレーニング済みのモジュールをトレーニングすることは、正則化を行っても埋め込みの重みが多様なデータでトレーニングされた言語モデルを表現することができなくなるという意味で危険です。その代わりに収束して新しいデータセットを理想的な表現にします。